In [1]:
!pip install -q torch torchinfo bitsandbytes peft trl accelerate datasets transformers sentencepiece evaluate scikit-learn

In [1]:
import torch
from torchinfo import summary
from datasets import load_dataset
import sys
sys.path.append('../')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
from transformers import DistilBertTokenizer
from MDMT_DistilBert import MDMT_DistilBertConfig, MDMT_DistilBertWrapper

config = MDMT_DistilBertConfig(tasks_configs=[dict(target_features=1, criterion_type='BCE')], max_position_embeddings=4096)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = MDMT_DistilBertWrapper.from_pretrained("distilbert-base-uncased", config=config, ignore_mismatched_sizes=True).to(device)
model.base_model.resize_position_embeddings(4096)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of MDMT_DistilBertWrapper were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifiers.0.bias', 'classifiers.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of MDMT_DistilBertWrapper were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized because the shapes did not match:
- distilbert.embeddings.position_embeddings.weight: found shape torch.Size([512, 768]) in the checkpoint and torch.Size([4096, 768]) in the

In [4]:
text = "Quote: Imagination is more"
inputs = tokenizer(text, return_tensors="pt").to(device)

model(**inputs).logits
print(summary(model, input_data=[inputs.input_ids, inputs.attention_mask]))

Layer (type:depth-idx)                                  Output Shape              Param #
MDMT_DistilBertWrapper                                  [1, 1]                    --
├─DistilBertModel: 1-1                                  [1, 7, 768]               --
│    └─Embeddings: 2-1                                  [1, 7, 768]               --
│    │    └─Embedding: 3-1                              [1, 7, 768]               23,440,896
│    │    └─Embedding: 3-2                              [1, 7, 768]               1,572,864
│    │    └─LayerNorm: 3-3                              [1, 7, 768]               1,536
│    │    └─Dropout: 3-4                                [1, 7, 768]               --
│    └─Transformer: 2-2                                 [1, 7, 768]               --
│    │    └─ModuleList: 3-5                             --                        42,527,232
├─ModuleList: 1-2                                       --                        --
│    └─Linear: 2-3                

In [2]:
class MTMD_Dataset():
    def __init__(self, dataset, task_index, text: str, labels: list[str], label_collator = lambda x: x):
        self.dataset = dataset
        self.task_index = task_index
        self.text = text
        self.labels = labels
        self.label_collator = label_collator
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        item["task_index"] = self.task_index
        item["full_text"] = item[self.text]
        item["labels"] = [self.label_collator(item[label]) for label in self.labels]
        return item

In [3]:
LLM_Detect_dataset = MTMD_Dataset(load_dataset("csv", data_files="Training_Essay_Data.csv", split="train"), task_index=0, text="text", labels=["generated"])

In [4]:
LLM_Detect_train_dataset, LLM_Detect_eval_dataset = torch.utils.data.random_split(LLM_Detect_dataset, [0.999, 0.001])
print(len(LLM_Detect_train_dataset), len(LLM_Detect_eval_dataset))

29116 29


In [ ]:
AI_Text_dataset = MTMD_Dataset(load_dataset("artem9k/ai-text-detection-pile", split="train"), task_index=0, text="text", labels=["source"], label_collator = lambda label: 0 if label == 'human' else 1)
datasets = [AI_Text_dataset]

loss_weights = torch.nn.functional.normalize(1/torch.tensor([len(dataset) for dataset in datasets], device=device, dtype=torch.float), p=1.0, dim=0)
print(f"dataset lenghts: {[len(dataset) for dataset in datasets]}, loss weights: {loss_weights.data}")

In [ ]:
_, AI_Text_train_dataset, AI_Text_eval_dataset = torch.utils.data.random_split(AI_Text_dataset, [0.95, 0.045, 0.005])
print(len(AI_Text_train_dataset), len(AI_Text_eval_dataset))

62664 6962


In [9]:
def collate_fn(batch):
    full_texts = [example["full_text"] for example in batch]
    labels = torch.stack([torch.stack([torch.tensor(label, device=device, dtype=torch.float32) for label in example["labels"]]) for example in batch]) if "labels" in batch[0] else None
    task_indices = torch.tensor([example["task_index"] for example in batch], device=device, dtype=torch.long) if "task_index" in batch[0] else None
    encoded = tokenizer(full_texts, return_tensors='pt', padding='longest', truncation=True, max_length=config.max_position_embeddings)
    output = { "input_ids": encoded["input_ids"], 
               "attention_mask": encoded["attention_mask"], 
               "labels": labels, 
               "task_indices": task_indices,
               "loss_weights": loss_weights }
    return output

In [14]:
import transformers

loss_weights = torch.tensor([1.0], device=device, dtype=torch.float)

def preprocess_logits(logits, _):
    return torch.round(logits, decimals=0)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    
    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

trainer = transformers.Trainer(
    model=model,
    train_dataset=LLM_Detect_train_dataset,
    eval_dataset=LLM_Detect_eval_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        warmup_ratio=0.05,
        num_train_epochs=1,
        learning_rate=1e-5,
        output_dir="outputs",
        optim="adamw_torch",
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        logging_steps=100//16,
        prediction_loss_only=False,
        save_strategy="no",
    ),
    preprocess_logits_for_metrics=preprocess_logits,
    compute_metrics=compute_metrics,
    data_collator=collate_fn
)

In [ ]:
trainer.train()

In [15]:
trainer.evaluate()

{'eval_loss': 0.017326263710856438,
 'eval_accuracy': 0.9975978037062457,
 'eval_precision': 0.9975984811719034,
 'eval_recall': 0.9975978037062457,
 'eval_f1': 0.9975980175790605,
 'eval_runtime': 42.9035,
 'eval_samples_per_second': 67.92,
 'eval_steps_per_second': 8.507}

In [14]:
trainer.evaluate(LLM_Detect_eval_dataset)

{'eval_loss': 2.341658353805542,
 'eval_accuracy': 0.5178448867536033,
 'eval_precision': 0.53795704297252,
 'eval_recall': 0.5178448867536033,
 'eval_f1': 0.523430779214353,
 'eval_runtime': 43.3632,
 'eval_samples_per_second': 67.2,
 'eval_steps_per_second': 8.417,
 'epoch': 0.9998723349929785}

In [5]:
from DetectGPT.model import GPT2PPLV2 as GPT2PPL

detectGPT = GPT2PPL()

/home/tommidi/Projects/NLP_Project/AI_Content_Detection/DetectGPT/model.py:63: SyntaxWarning: invalid escape sequence '\d'
  tokens = list(re.finditer("<extra_id_\d+>", text))
/home/tommidi/Projects/NLP_Project/AI_Content_Detection/DetectGPT/model.py:87: SyntaxWarning: invalid escape sequence '\d'
  pattern = re.compile("<extra_id_\d+>")
/home/tommidi/Projects/NLP_Project/AI_Content_Detection/DetectGPT/model.py:159: SyntaxWarning: invalid escape sequence '\d'
  texts = list(re.finditer("[^\d\W]+", original_text))
/home/tommidi/Projects/NLP_Project/AI_Content_Detection/DetectGPT/model.py:195: SyntaxWarning: invalid escape sequence '\d'
  sentence_length = len(list(re.finditer("[^\d\W]+", sentence)))
/home/tommidi/Projects/NLP_Project/AI_Content_Detection/DetectGPT/model.py:222: SyntaxWarning: invalid escape sequence '\['
  sentence = re.sub("\[[0-9]+\]", "", sentence) # remove all the [numbers] cause of wiki


/home/tommidi/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def collate_fn_detectGPT(batch):
    full_texts = [example["full_text"] for example in batch]
    labels = torch.stack([torch.stack([torch.tensor(label, device=device, dtype=torch.float32) for label in example["labels"]]) for example in batch]) if "labels" in batch[0] else None
    output = { "text": full_texts,
               "chunk_value": 100,
               "labels": labels }
    return output

In [7]:
import transformers

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics_detectGPT(pred):
    labels = pred.label_ids
    preds = pred.predictions
    
    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

trainer_detectGPT = transformers.Trainer(
    model=detectGPT,
    train_dataset=LLM_Detect_train_dataset,
    eval_dataset=LLM_Detect_eval_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        warmup_ratio=0.05,
        num_train_epochs=1,
        learning_rate=1e-5,
        output_dir="outputs",
        optim="adamw_torch",
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        logging_steps=100//16,
        prediction_loss_only=False,
        save_strategy="no",
        label_names=["labels"]
    ),
    compute_metrics=compute_metrics_detectGPT,
    data_collator=collate_fn_detectGPT
)

In [8]:
trainer_detectGPT.evaluate()

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], device='cuda:0')


tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.]], device='cuda:0')
tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], device='cuda:0')
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], device='cuda:0')


{'eval_loss': 0.0,
 'eval_accuracy': 0.7931034482758621,
 'eval_precision': 0.8390804597701149,
 'eval_recall': 0.7931034482758621,
 'eval_f1': 0.7439655172413793,
 'eval_runtime': 215.9717,
 'eval_samples_per_second': 0.134,
 'eval_steps_per_second': 0.019}